In [ ]:
-- --TABLEAU_INTERNAL_API_TOKEN
-- use schema PYTHON_WORKLOADS.TABLEAU_ADMIN;
-- CREATE OR REPLACE function get_secret_type_1()
-- RETURNS STRING
-- LANGUAGE PYTHON
-- RUNTIME_VERSION = 3.9
-- HANDLER = 'get_secret_1'
-- EXTERNAL_ACCESS_INTEGRATIONS = (tableau_internal_prd_apis_access_integration)
-- SECRETS = ('cred' = TABLEAU_INTERNAL_API_TOKEN )
-- AS
-- $$
-- import _snowflake
-- def get_secret_1():
--   secret_type = _snowflake.get_generic_secret_string('cred')
--   return secret_type
-- $$;
--  grant usage on function get_secret_type_1() to role svc_engineer_prd;

In [ ]:
-- create or replace table bi.stage.new_tableau_users as 
-- select
--     r.personoid,
--     --lower(regexp_replace(e.workemail, '@.*', '')) as username,
--     lower(e.workemail) as username,
--     concat(e.lastname, ', ', e.firstname) as name,
--     'Viewer' as role_level,
--     null as tableau_user_id,
--     current_timestamp as create_date,
--     current_timestamp as update_date
-- from
--     bi.sales.current_role_type r
--     left join bi.sales.employees e on r.personoid = e.personoid
--     left join reference.tableau.all_users a on r.personoid = a.personoid
--     join (
--         select
--             distinct personoid
--         from
--             hcsdb.dbo.somanagerarea
--         where
--             current_date between datebegin
--             and dateend
--     ) p on r.personoid = p.personoid
--     left join bi.sales.current_channel c on r.personoid = c.personoid
--     left join bi.sales.current_status cs on r.personoid = cs.personoid
-- where
--     a.tableau_user_id is null
--     and r.role_type in (2, 3, 4)
--     and e.terminationdate is null
--     and ifnull(cs.status, 'None') <> 'Other'
--     and c.channel <> 'HSC/Other'
--     and e.jobdescription not like '%HSC%';
    
-- create or replace table bi.stage.new_tableau_users_groups as 
-- select
--     r.personoid,
--     '2c7ff875-3164-4598-9cc9-348aca03d30c' as group_id,
--     'Sales_Viewer' as group_name,
--     current_date as begin_date,
--     '2050-12-31' as end_date,
--     current_timestamp as create_date
-- from
--     bi.sales.current_role_type r
--     left join bi.sales.employees e on r.personoid = e.personoid
--     left join reference.tableau.all_users a on r.personoid = a.personoid
--     join (
--         select
--             distinct personoid
--         from
--             hcsdb.dbo.somanagerarea
--         where
--             current_date between datebegin
--             and dateend
--     ) p on r.personoid = p.personoid
--     left join bi.sales.current_channel c on r.personoid = c.personoid
--     left join bi.sales.current_status cs on r.personoid = cs.personoid
-- where
--     a.tableau_user_id is null
--     and r.role_type in (2, 3, 4)
--     and e.terminationdate is null
--     and ifnull(cs.status, 'None') <> 'Other'
--     and c.channel <> 'HSC/Other'
--     and e.jobdescription not like '%HSC%'
    
-- union all

-- select
--     r.personoid,
--     'ead914e6-e876-4e6f-9a36-cd87dccf98e6' as group_id,
--     'Sales_DM_Viewer' as group_name,
--     current_date as begin_date,
--     '2050-12-31' as end_date,
--     current_timestamp as create_date
-- from
--     bi.sales.current_role_type r
--     left join bi.sales.employees e on r.personoid = e.personoid
--     left join reference.tableau.all_users a on r.personoid = a.personoid
--     join (
--         select
--             distinct personoid
--         from
--             hcsdb.dbo.somanagerarea
--         where
--             current_date between datebegin
--             and dateend
--     ) p on r.personoid = p.personoid
--     left join bi.sales.current_channel c on r.personoid = c.personoid
--     left join bi.sales.current_status cs on r.personoid = cs.personoid
-- where
--     a.tableau_user_id is null
--     and r.role_type in (2)
--     and e.terminationdate is null
--     and ifnull(cs.status, 'None') <> 'Other'
--     and c.channel <> 'HSC/Other'
--     and e.jobdescription not like '%HSC%';
  
  
  
  
-- merge into reference.tableau.all_users s 
-- using bi.stage.new_tableau_users t on s.personoid = t.personoid
-- when not matched then insert (
--     personoid,
--     username,
--     name,
--     role_level,
--     tableau_user_id,
--     create_date,
--     update_date
-- ) values (
--     t.personoid,
--     t.username,
--     t.name,
--     'Viewer',
--     null,
--     current_timestamp,
--     current_timestamp
-- );

-- merge into reference.tableau.user_group_detail s 
-- using bi.stage.new_tableau_users_groups t on s.personoid = t.personoid
-- when not matched then insert (
--     personoid,
--     group_id,
--     group_name,
--     begin_date,
--     end_date,
--     create_date
-- ) values (
--     t.personoid,
--     t.group_id,
--     t.group_name,
--     t.begin_date,
--     t.end_date,
--     current_timestamp
-- );

In [ ]:
from snowflake.snowpark.context import get_active_session
import pandas as pd
import requests
import json
import datetime
import tableauserverclient as TSC
from tableauserverclient import TableauAuth, Server
import tempfile

TABLEAU_ENVIRONMENT = 'prod'

# Snowflake session initialization
session = get_active_session()

api_secret_query = "select PYTHON_WORKLOADS.TABLEAU_ADMIN.get_secret_type_1()"
TABLEAU_PAT_SECRET = session.sql(api_secret_query).collect()[0][0]

# Snowflake configuration
session.use_warehouse('WH_ENGINEER')
session.use_database('REFERENCE')
session.use_role('SVC_ENGINEER_PRD')

# Tableau configuration
if TABLEAU_ENVIRONMENT == 'prod':
    TABLEAU_API_VERSION = '3.19'
    TABLEAU_PAT_NAME = 'matillionautomation'

    TABLEAU_SERVER = 'https://tableau.heartland.us'
    TABLEAU_ADMIN_USER = 'prakharsingh.sengar@e-hps.com'
    Z_SOURCE_SCHEMA = 'TABLEAU'
    Z_SOURCE_TABLE =  'ALL_USERS'
    
    #TABLEAU_ADMIN_PASS = ''


else:
    print('Invalid Tableau Creds')
    sys.exit()

# Tableau Server connection using PAT
tableau_auth = TSC.PersonalAccessTokenAuth(TABLEAU_PAT_NAME, TABLEAU_PAT_SECRET, site_id='')
server = TSC.Server(TABLEAU_SERVER, use_server_version=True)
server.auth.sign_in(tableau_auth)

# Create event function
def createEvent(email, event, event_status, event_stage, event_description):
    sql = f"insert into reference.tableau.user_management_event_history values ({email}, '{event}', '{event_status}', '{event_stage}', {event_description}, '{datetime.datetime.now()}')"
    session.sql(sql).collect()

STAGES = {
    0: 'COULD NOT CREATE USER IN TABLEAU',
    1: 'COULD NOT UPDATE USER IN TABLEAU',
    2: 'COULD NOT UPDATE TABLEAU_USER_ID IN SNOWFLAKE',
    3: 'COULD NOT ADD USER TO TABLEAU GROUPS',
    4: 'USER FULLY CREATED'
}

# Fetch users to be processed
sql = f"select u.* from {Z_SOURCE_SCHEMA}.{Z_SOURCE_TABLE} u where tableau_user_id is null or tableau_user_id = ''"
rows = session.sql(sql).collect()

for row in rows:
    PERSONOID = row[0]
    EMAIL_ADDRESS = row[1].strip()
    FULL_NAME = row[2]
    SITE_ROLE = row[3]
    USER_PASSWORD = 'TemporaryPassword123!'
    STAGE = 0
    SUCCESS = 'Success'
    ERROR = ''
    try:
        # Create user in Tableau
        new_user = TSC.UserItem(EMAIL_ADDRESS, SITE_ROLE)
        new_user = server.users.add(new_user)
        TABLEAU_USER_ID = new_user.id
        STAGE += 1
        
        # Update user in Tableau
        new_user.email = EMAIL_ADDRESS
        new_user.fullname = FULL_NAME
        server.users.update(new_user)
        STAGE += 1
        
        # Update Tableau user ID in Snowflake
        update_sql = f"update reference.tableau.all_users set tableau_user_id = '{TABLEAU_USER_ID}' where username = '{EMAIL_ADDRESS}'"
        session.sql(update_sql).collect()
        STAGE += 1
        
        # Add user to Tableau groups
        #group_sql = f"select distinct group_id from reference.tableau.user_group_detail where personoid = {PERSONOID}"
        group_sql = f"select distinct GROUP_NAME from reference.tableau.user_group_detail where personoid = {PERSONOID}"
        group_rows = session.sql(group_sql).collect()
        all_groups, pagination_item = server.groups.get()
        for group_row in group_rows:
            GROUP_NAME = group_row[0]
        #     GROUP_ID = group_row[0]
        #     server.groups.add_user(GROUP_ID, TABLEAU_USER_ID)
            for group in all_groups:
                        if group.name in GROUP_NAME:
                            try:
                                print(f"Adding user to group: {group.name} (ID: {group.id})")
                                server.groups.add_user(group, TABLEAU_USER_ID)
                            except Exception as e:
                                print(f"Failed to add user to group {group.name} (ID: {group.id}): {e}")
        STAGE += 1
        
        print(f'Successfully added user {EMAIL_ADDRESS}')
    except Exception as e:
        SUCCESS = 'Failed'
        ERROR = str(e)
        print(f'Failed to add user {EMAIL_ADDRESS}, {STAGES[STAGE]}')
    
    ERROR_updated = "$$" + ERROR + "$$"
    EMAIL_ADDRESS = "$$" + EMAIL_ADDRESS + "$$"
    createEvent(EMAIL_ADDRESS, 'User creation', SUCCESS, STAGES[STAGE], ERROR_updated)

# Sign out from Tableau server
server.auth.sign_out()

In [ ]:
# from snowflake.snowpark.context import get_active_session
# import requests
# import pandas as pd

# # Snowflake session initialization
# session = get_active_session()

# api_secret_query = "select PYTHON_WORKLOADS.TABLEAU_ADMIN.get_secret_sendgrid()"
# sendgrid_api_key = session.sql(api_secret_query).collect()[0][0]


# # Run the SHOW USERS command
# session.sql("use role securityadmin")
# show_users_rows = session.sql("SHOW USERS").collect()

# # Convert the list of rows to a pandas DataFrame manually
# #show_users_df = pd.DataFrame([row.as_dict() for row in show_users_rows])

# # Query to filter inactive users who haven't logged in for the past 85 days
# inactive_users_rows = session.sql("""
#     SELECT "name", "login_name", "display_name", "first_name", "last_name", "email", "disabled", "default_warehouse", 
#            "default_role", "ext_authn_duo", "owner", "last_success_login", "has_password", "has_rsa_public_key"
#     FROM table(RESULT_SCAN(last_query_id()))
#     WHERE "disabled" =  'false'
#     AND "last_success_login" < current_date-85
#     ORDER BY "last_success_login" DESC
# """).collect()

# inactive_users_df = pd.DataFrame([row.as_dict() for row in inactive_users_rows])

# # Retrieve the SendGrid API key from the Snowflake secret
# #sendgrid_api_key = session.sql("SELECT DB.SCHEMA.GET_SENDGRID_SECRET() AS SECRET").collect()[0]['SECRET']

# # Email settings
# email_subject_template = "Action Required: Snowflake User '{login_name}' Inactivity"
# email_body_template = """
# Dear Snowflake User,

# Our records indicate that you haven't logged into Snowflake account as user '{login_name}' in the last 85 days.
# To ensure you have continued access to your Snowflake resources, please log in as soon as possible.

# If you do not log in within the next four days, your user account will be deactivated as per our security policy.

# Best regards,
# BI Team
# """

# headers = {
#     'Authorization': f'Bearer {sendgrid_api_key}',
#     'Content-Type': 'application/json'
# }

# # CC Emails
# cc_emails = ['william.cummins@e-hps.com', 'jerrod.stutzman@e-hps.com', 'prakhar.sengar@heartland.us', 'justin.lewis@e-hps.com', 'dmiddleton@tsys.com']
# #cc_emails = ['prakhar.sengar@heartland.us']

# # Send an email to each inactive user
# for index, row in inactive_users_df.iterrows():
#     email_subject = email_subject_template.format(login_name=row['login_name'])
#     email_body = email_body_template.format(first_name=row['first_name'], last_name=row['last_name'], login_name=row['login_name'])
    
#     data = {
#         "personalizations": [{
#             "to": [{"email": row['email']}],
#             #"to": [{"email": "prakharsingh.sengar@e-hps.com"}],
#             "cc": [{"email": cc} for cc in cc_emails],
#             "subject": email_subject
#         }],
#         "from": {"email": "BI@heartland.us"},
#         "content": [{"type": "text/plain", "value": email_body}]
#     }
    
#     response = requests.post('https://api.sendgrid.com/v3/mail/send', headers=headers, json=data)
    
#     if response.status_code == 202:
#         print(f"Email sent successfully to {row['email']} for account {row['login_name']}")
#     else:
#         print(f"Failed to send email to {row['email']} for account {row['login_name']}. Error: {response.text}")

# # Conclusion
# print("All emails have been processed.")
